In [125]:
import duckdb as ddb

con = ddb.connect(database=":memory:")

con.sql("""
CREATE TABLE program_beslut AS
    FROM 'data/resultat-ansokningsomgang-2020-2024-beslut.csv';
CREATE TABLE program_kommun AS
    FROM 'data/resultat-ansokningsomgang-2020-2024-diarie_kommun.csv';

CREATE TABLE kurser_beslut AS
    FROM 'data/resultat-for-kurser-inom-yh-2024-beslut.csv';
CREATE TABLE kurser_kommun AS
    FROM 'data/resultat-for-kurser-inom-yh-2024-diarie_kommun.csv';
""")

In [126]:
con.sql("""select * from kurser_beslut""").describe()

┌─────────┬───────────────┬──────────────────┬─────────┬─────────────────────────────────┬─────────────────────────────────────────────────┬───────────────────┬───────────────────┬────────────────────┬──────────────────────────┬────────────────────┬──────────────┬───────────┬────────────────────┬────────────────┐
│  aggr   │ Diarienummer  │ Ansökningsomgång │ Beslut  │      Utbildningsanordnare       │                 Utbildningsnamn                 │ Utbildningsområde │     YH-poäng      │ Beviljade platser  │ Beviljade platser totalt │       Start        │     Län      │  Kommun   │   Antal kommuner   │ Flera kommuner │
│ varchar │    varchar    │      double      │ varchar │             varchar             │                     varchar                     │      varchar      │      double       │       double       │          double          │       double       │   varchar    │  varchar  │       double       │    varchar     │
├─────────┼───────────────┼──────────────────┼─────────

In [127]:
def get_utbildningsomraden(con):
    rel_utbildningsomrade = con.sql("""
    select distinct Utbildningsområde
    from kurser_beslut
    order by Utbildningsområde
    """)
    return [i[0] for i in rel_utbildningsomrade.fetchall()]

get_utbildningsomraden(con)

['Data/IT',
 'Ekonomi, administration och försäljning',
 'Friskvård och kroppsvård',
 'Hotell, restaurang och turism',
 'Hälso- och sjukvård samt socialt arbete',
 'Journalistik och information',
 'Juridik',
 'Kultur, media och design',
 'Lantbruk, djurvård, trädgård, skog och fiske',
 'Pedagogik och undervisning',
 'Samhällsbyggnad och byggteknik',
 'Säkerhetstjänster',
 'Teknik och tillverkning',
 'Transporttjänster']

In [131]:
def create_df_beviljad(con, year=2024):
    rel_beviljad = con.sql("""
    select
        Utbildningsområde,
        count(*) as Beviljad
    from kurser_beslut
    where
        Beslut = true
        and Ansökningsomgång = $year
    group by Utbildningsområde
    order by Beviljad desc, Utbildningsområde
    ;""",
    params={"year": year})
    return rel_beviljad.df().set_index("Utbildningsområde")

create_df_beviljad(con)

,Beviljad
Utbildningsområde,
"Ekonomi, administration och försäljning",171
Data/IT,111
Teknik och tillverkning,106
Samhällsbyggnad och byggteknik,93
Hälso- och sjukvård samt socialt arbete,81
"Hotell, restaurang och turism",19
"Lantbruk, djurvård, trädgård, skog och fiske",18
Pedagogik och undervisning,15
Journalistik och information,11


In [132]:
def create_df_avslag(con, year=2024):
    rel_avslag = con.sql("""
    select
        Utbildningsområde,
        count(*) as Avslag
    from kurser_beslut
    where
        Beslut = false
        and Ansökningsomgång = $year
    group by Utbildningsområde
    order by Avslag desc, Utbildningsområde
    ;""",
    params={"year": year})
    return rel_avslag.df().set_index("Utbildningsområde")

create_df_avslag(con)

,Avslag
Utbildningsområde,
"Ekonomi, administration och försäljning",112
Data/IT,84
Hälso- och sjukvård samt socialt arbete,53
Teknik och tillverkning,51
Samhällsbyggnad och byggteknik,41
"Lantbruk, djurvård, trädgård, skog och fiske",24
Pedagogik och undervisning,18
Säkerhetstjänster,11
"Hotell, restaurang och turism",9


In [130]:
def get_antal_kurs(con):
    rel_antal = con.sql("""
    select
        (select count(*) from kurser_beslut) as "Antal kurs-ansökningar",
        (select count(*) from kurser_kommun) as "Antal kurs-ansökningar, alla kommuner"
    """)
    return rel_antal.fetchone()

get_antal_kurs(con)

(1090, 1202)